# Recuperation des données depuis le WEB

In [2]:
import requests
import os
from datetime import datetime

# URL du fichier à télécharger
url = "https://www.insee.fr/fr/statistiques/fichier/2120875/int_courts_naf_rev_2.xls"

# Créer le dossier 'data' s'il n'existe pas
if not os.path.exists('data'):
    os.makedirs('data')

# Télécharger le fichier
try:
    response = requests.get(url)
    response.raise_for_status()  # Vérifie si la requête a réussi
    
    # Générer le nom du fichier avec la date
    date_str = datetime.now().strftime("%Y%m%d")
    filename = f"data/naf_rev2_{date_str}.xls"
    
    # Sauvegarder le fichier
    with open(filename, 'wb') as file:
        file.write(response.content)
    
    print(f"Le fichier a été téléchargé avec succès et sauvegardé sous: {filename}")

except requests.exceptions.RequestException as e:
    print(f"Erreur lors du téléchargement du fichier: {e}")
except IOError as e:
    print(f"Erreur lors de l'écriture du fichier: {e}")


Le fichier a été téléchargé avec succès et sauvegardé sous: data/naf_rev2_20250129.xls


# Envoie du fichier local vers le Data Lake 

In [ ]:
from azure.storage.filedatalake import DataLakeServiceClient
import os
from datetime import datetime

# Configuration
storage_account_name = "datalakecertifimpe"
storage_account_key = "xxxxxxxxxxxxxxxxxxxxx"
container_name = "data-gouv"

# Construire l'URL du service
service_client = DataLakeServiceClient(
    account_url=f"https://{storage_account_name}.dfs.core.windows.net",
    credential=storage_account_key
)

def upload_to_datalake(local_file_path, remote_folder_path):
    try:
        # Obtenir une référence au système de fichiers (container)
        file_system_client = service_client.get_file_system_client(file_system=container_name)
        
        # Obtenir le nom du fichier à partir du chemin local
        file_name = os.path.basename(local_file_path)
        
        # Construire le chemin distant complet
        remote_file_path = f"{remote_folder_path}/{file_name}"
        
        # Créer un client pour le fichier
        file_client = file_system_client.get_file_client(remote_file_path)
        
        # Ouvrir et uploader le fichier
        with open(local_file_path, 'rb') as file:
            file_contents = file.read()
            file_client.upload_data(file_contents, overwrite=True)
            
        print(f"Fichier uploadé avec succès vers: {remote_file_path}")
        
    except Exception as e:
        print(f"Une erreur s'est produite: {str(e)}")

# Utilisation de la fonction
if __name__ == "__main__":
    # Chemin du fichier local (ajuster selon votre structure)
    date_str = datetime.now().strftime("%Y%m%d")
    local_file = f"data/naf_rev2_{date_str}.xls"
    
    # Chemin dans le Data Lake
    remote_folder = "data/insee/naf_rev2"
    
    # Upload du fichier
    if os.path.exists(local_file):
        upload_to_datalake(local_file, remote_folder)
    else:
        print(f"Le fichier local {local_file} n'existe pas")


Fichier uploadé avec succès vers: data/insee/naf_rev2/naf_rev2_20250129.xls
